# PyCOMPSs: Using objects, lists, and synchronization

In this example we will see how classes and objects can be used from PyCOMPSs, and that class methods can become tasks.

## Import the PyCOMPSs library

In [ ]:
import pycompss.interactive as ipycompss

## Start the runtime  
Initialize COMPSs runtime
Parameters indicates if the execution will generate task graph, tracefile, monitor interval and debug information. 

In [ ]:
ipycompss.start(graph=True, monitor=1000, debug=True)
#trace=True

## Importing task and arguments directionality modules 
Import task module before annotating functions or methods 

In [ ]:
from pycompss.api.api import compss_barrier
from pycompss.api.api import compss_wait_on
from pycompss.api.task import task

## Declaring a class

In [ ]:
%%writefile my_shaper.py 

from pycompss.api.task import task
from pycompss.api.parameter import IN

class Shape(object):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        description = "This shape has not been described yet"

    @task(returns=int)
    def area(self):
        return self.x * self.y
    
    @task(returns=int)
    def perimeter(self):
        return 2 * self.x + 2 * self.y
    
    def describe(self,text):
        self.description = text
        
    @task()    
    def scaleSize(self,scale):
        self.x = self.x * scale
        self.y = self.y * scale
        
    @task(target_direction=IN)
    def infoShape(self):
        print('Shape x=', self.x, 'y= ', self.y)
        

In [ ]:
@task(returns=int)
def addAll(*mylist):
    sum = 0
    for ll in mylist:
        sum = sum + ll
    return sum    

## Invoking tasks 

In [ ]:
from my_shaper import Shape

In [ ]:
my_shapes = []
my_shapes.append(Shape(100,45))
my_shapes.append(Shape(50,50))
my_shapes.append(Shape(10,100))
my_shapes.append(Shape(20,30))

In [ ]:
all_areas = []

In [ ]:
for this_shape in my_shapes:
    all_areas.append(this_shape.area())

In [ ]:
# Need it if we want to synchonize nested objects
all_areas = compss_wait_on(all_areas)
print(all_areas)

In [ ]:
rectangle = Shape(200,25)
rectangle.scaleSize(5)
area_rectangle = rectangle.area()
rectangle = compss_wait_on(rectangle)
print('X = %d' % rectangle.x)
area_rectangle = compss_wait_on(area_rectangle)
print('Area = %d' % area_rectangle)

In [ ]:
all_perimeters=[]
my_shapes.append(rectangle)
for this_shape in my_shapes:
    this_shape.infoShape()
    all_perimeters.append(this_shape.perimeter())

In [ ]:
# all_perimeters = compss_wait_on(all_perimeters)
# print all_perimeters

In [ ]:
mysum = addAll(*all_perimeters)
mysum = compss_wait_on(mysum)
print(mysum)

## Stop the runtime

In [ ]:
ipycompss.stop(sync=True)

#### <font color=green> **Remember:** *The Python kernel must be restarted before starting again PyCOMPSs within the same notebook.* </font>